In [ ]:
import requests
import pandas as pd
from datetime import datetime

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "5364c78eab167c6b826b39227030cfa4"  # Replace with your actual API key


provinces = {
    "Chiang Mai":{
        "lat": 14.0134,
        "lon": 98.6601
    },
    "Lamphun":{
        "lat": 18.5745,
        "lon": 99.0087
    },
    "Lampang":{
        "lat": 17.6289,
        "lon": 100.0977
    },
    "Uttaradit":{
        "lat": 17.6255,
        "lon": 100.0942
    },
    "Phrae":{
        "lat": 18.1459,
        "lon": 100.1410
    },
    "Nan":{
        "lat": 18.7838,
        "lon": 100.7789
    },
    "Phayao":{
        "lat": 19.1920,
        "lon": 99.8788
    },
    "Chiang Rai":{
        "lat": 19.9086 ,
        "lon": 99.8325
    },
    "Mae Hong Son":{
        "lat": 19.3006 ,
        "lon": 97.9686
    }
}

# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        timestamp = datetime.fromtimestamp(data['dt'])
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [ ]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

In [ ]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
lakefs_endpoint = "http://lakefs-dev:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

In [ ]:
from datetime import datetime
import pytz

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)